In [ ]:
import pandas as pd
import numpy as np
import vectorbt as vbt
import MetaTrader5 as mt5
import datetime
import time


if not mt5.initialize():
    print("initialize() failed, error code =", mt5.last_error())
    quit()
symbol = 'XAUUSD'  # or any other valid symbol
symbol_info = mt5.symbol_info(symbol)

while True:
    print("Getting data in progress...")
    # Get the latest data
    bars = mt5.copy_rates_from(symbol, mt5.TIMEFRAME_M1, datetime.datetime.now(), 365)
    df = pd.DataFrame(bars)
    df['time'] = pd.to_datetime(df['time'], unit='s')
    df = df.set_index('time')

    # Calculate RSI
    rsi = vbt.RSI.run(df['close'], 14)
    
    # Check buy condition
    buy_condition = rsi.rsi > 78
    
    # Check sell condition
    sell_condition = rsi.rsi < 22

    
    if buy_condition.iloc[-1]:
        print("Buy condition met:")
        current_price = df['close'].iloc[-1]
        stop_loss = current_price - 1000 * symbol_info.point
        take_profit = current_price + 1500* symbol_info.point
        print(f"RSI: {rsi.rsi.iloc[-1]:.2f} (above 78)")
        print(f"Current price: {current_price:.5f}")
        print(f"Stop loss: {stop_loss:.5f}")
        print(f"Take profit: {take_profit:.5f}")
        print("-" * 30)
    
    if sell_condition.iloc[-1]:
        print("Sell condition met:")
        current_price = df['close'].iloc[-1]
        stop_loss = current_price + 1500 * symbol_info.point
        take_profit = current_price - 1000* symbol_info.point
        print(f"RSI: {rsi.rsi.iloc[-1]:.2f} (below 22)")
        print(f"Current price: {current_price:.5f}")
        print(f"Stop loss: {stop_loss:.5f}")
        print(f"Take profit: {take_profit:.5f}")
        print("-" * 30)

    time.sleep(60)  # wait for 60 seconds